In [37]:
'''
Code to improve my bot: https://github.com/Norbaeocystin/FxBot
Date: December 2018
Author: Rastislav Baran
Usage: to improve profitable dates generated by bot

Here are few potential problems to make analysis less profitable trades than unprofitable

Example of usage:

X, Y = get_structured_data(3600)

rnn = NNModel()
rnn.fit(X,Y, generator = True)

x, y = generate_random_dataset(X,Y)
predicted = rnn.sess.run(rnn.predictions, feed_dict = {rnn.X: x} )
#for all dataseet
X_normalized = normalize_maxmin(X)
predicted = rnn.sess.run(rnn.predictions, feed_dict = {rnn.X: X} )
'''
import datetime
import logging
import numpy as np
import pandas as pd
import pymongo
from pymongo import MongoClient
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
import tensorflow as tf
import time

# Connectio URI can be in shape mongodb://<username>:<password>@<ip>:<port>/<authenticationDatabase>')
CONNECTION = MongoClient('localhost')
db = CONNECTION.Bot
FxData = db.FxData

logging.basicConfig(level=logging.DEBUG, format = '%(asctime)s %(name)s %(levelname)s %(message)s')
logger = logging.getLogger(__name__)



def get_info_about_trades():
    '''
    returns: <dict>, profitable trades and unprofitable with sum profits and losses as dict
    '''
    return FxData.aggregate([{'$facet':{'loss':[{ '$match': { 'order': { '$exists': True }, 'profit' :{'$lt':12} }},{'$count': "No"}], 
                                        'profitable':[{ '$match': { 'order': { '$exists': True }, 'profit' :{'$gt':12} }},{'$count': "No"}], 
                                        'PL':[{ '$match': { 'order': { '$exists': True } }},{ '$group': { '_id' : None, 'sum' : { '$sum': "$profit" }}}]}
                             }]).next()

def get_trades():
    '''
    function which retuns list of dictionaries with data about closed trades
    returns: <list>, list of dictionaries with data about closed trades
    '''
    return list(FxData.aggregate([{ '$match': { 'order': { '$exists': True }}}]))

def get_structured_data(time = 100, limit = 0):
    '''
    Function to get structured data from MongoDB, please it can take time ...
    returns: <tuple>, tuple: <numpy.array>, <numpy.array> or if you want: features, labels
        
    args:
        time: <int>, how long features ( in this case timeserie) will be long in seconds
        limit: <int>, positive integer to limit number of trades if necessary
    '''
    trades = get_trades()
    if limit != 0 and limit > 0:
        trades = trades[:limit]
    #generate labels
    profits = [item.get('profit') for item in trades]
    logger.debug('Gets {} trades'.format(len(profits)))
    #convert list profits and losses to binary
    labels = np.array([[(0 if item < 0 else 1)] for item in profits]).astype(np.uint8)
    # features
    times = [item.get('Time') for item in trades]
    features = []
    logger.debug('Labels done. Working on features')
    # fetching data for features and structuring them
    for ind, item in enumerate(times):
        # lower or equal time that in item, only price to get, reversed it by sorting, limit size of data and reversed it again 
        x = list(FxData.find({'Time':{'$lte':item}},{"Price":1, '_id':0}).sort('Time', pymongo.DESCENDING).limit(time))[::-1]
        if len(x) != time:
            logger.debug('Data are shorter for item {}'.format(ind))
            break
        features.append([item.get('Price') for item in x])
        if ind % 10 == 0:
            logger.debug('Done structuring {} trades'.format(ind))
    features = np.array(features).astype(np.float32)
    logger.debug('Your data are prepared')
    return features, labels

def normalize_maxmin(X):
    '''
    MaxMinNormalizer
    return: <numpy.ndarray>, with values between 0 and 1
    
    args:
        X: <numpy.ndarray>, features, labels whatever
    '''
    return MinMaxScaler().fit_transform(X)

def generate_random_dataset(X, Y, ratio_of_profit = 0.7):
    '''
    one solution how to generate dataset with ration 50 % of profitable trades and loss trades
    return: <list>, list of shuffled features and labels as numpy.ndarray
    
    args:
        X: <numpy.ndarray>, features
        Y: <numpy.ndarray>, labels with shape (x, 1)
        ratio_of_profit: <float>, between 0 to 1, how many of profitable trades to use
    '''
    #returns indices of profitable trades
    indices_profitable = np.argwhere(Y == 1)[:,0]
    indices_profitable = np.random.choice(indices_profitable, size =  int(indices_profitable.shape[0]* ratio_of_profit))
    #returns profitable trades
    profitable_features = np.take(X,[indices_profitable], axis =0)[0]
    #now for loss trades
    indices_loss = np.argwhere(Y == 0)[:,0]
    indices_loss = np.random.choice(indices_loss, size =  int(indices_profitable.shape[0]* ratio_of_profit ))
    #returns profitable trades
    loss_features = np.take(X,[indices_loss], axis =0)[0]
    #concatenate
    features = np.concatenate((profitable_features, loss_features), axis = 0)
    profits = np.array([[1]] * profitable_features.shape[0])
    losses = np.array([[0]] * loss_features.shape[0])
    labels = np.concatenate([profits, losses], axis = 0)
    #shuffle dataset
    features = normalize_maxmin(features)
    return shuffle(features, labels)


class NNModel:
    
    def __init__(self, input_dim = 3600, output_dim = 1, learning_rate = 0.00001, dropout = 0.5):
        self.X  = tf.placeholder(tf.float32, shape = [None, input_dim], name = 'X')
        self.Y = tf.placeholder(tf.float32, shape = [None, output_dim], name = 'Y')
        self.logdir = 'logs/RNN_with_summaries'
        self.learning_rate = learning_rate
        self.dropout = dropout
        self.keep_prob = tf.placeholder_with_default(1.0, shape=())
        #NN
        '''
        posibility to add conv1d filters
        
        #input
        i = tf.constant([1, 0, 2, 3, 0, 1, 1,0], dtype=tf.float32, name='i')
        #kernel
        k = tf.constant([0, 1, 0], dtype=tf.float32, name='k')
        #reshaping
        data   = tf.reshape(i, [1, int(i.shape[0]), 1], name='data')
        kernel = tf.reshape(k, [int(k.shape[0]), 1, 1], name='kernel')
        #squeezing is neccessary to remove one dimension
        res = tf.squeeze(tf.nn.conv1d(data, kernel, 2, 'SAME'))
        '''
        self.fc1 = tf.contrib.layers.fully_connected(self.X, 2048, activation_fn = tf.nn.relu)
        self.d1 = tf.nn.dropout(self.fc1, self.keep_prob)
        self.fc2 = tf.contrib.layers.fully_connected(self.d1, 1024, activation_fn = tf.nn.relu) 
        self.fc3 = tf.contrib.layers.fully_connected(self.fc2, 1024, activation_fn = tf.nn.relu) 
        self.fc4 = tf.contrib.layers.fully_connected(self.fc3, 1024, activation_fn = tf.nn.relu) 
        self.d2 = tf.nn.dropout(self.fc4, self.keep_prob)
        self.fc5 = tf.contrib.layers.fully_connected(self.d2, 32, activation_fn = tf.nn.relu) 
        self.prediction = tf.layers.dense(self.fc5, 1, activation = tf.sigmoid)
        self.cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.prediction, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.cost)
        self.predicted_class = tf.greater(self.prediction,0.5)
        self.correct = tf.equal(self.predicted_class, tf.equal(self.Y,1.0))
        self.accuracy = tf.reduce_mean( tf.cast(self.correct, 'float') )
        self.init_op = tf.global_variables_initializer()
        self.config = tf.ConfigProto(device_count = {'GPU': 0})
        self.sess = tf.Session(config = self.config)
        self.accuracy_scalar = tf.summary.scalar('accuracy', self.accuracy)
    
    def fit(self, X, Y, epochs = 100000,every = 1000, generator = False, log = 'Forex_NN_log'):
        '''
        optimizes weights
        args:
          X: <numpy.array>, features 
          Y: <numpy.array>, labels
          epochs: <int>, number of epochs
          every: <int>, how often print message with Epoch and Loss values
          folder: <string>, name of folder where to store data for TensorBoard
          batch_size: <int>, size of features array, size of features sample
        '''
        self.sess.run(self.init_op)
        logger.debug('Initializing')
        summary_writer = tf.summary.FileWriter(log, self.sess.graph)
        for i in range(epochs):
            if generator:
                x, y = generate_random_dataset(X,Y)
                _, l,a, acc = self.sess.run([self.optimizer, self.cost,self.accuracy, self.accuracy_scalar], 
                                            feed_dict = {self.X: x, self.Y: y, self.keep_prob: self.dropout})
                summary_writer.add_summary(acc, i)
                if i % every == 0:
                    print('[ {} ] Epoch {} Loss: {:.7f} accuracy {}'.format(time.ctime(), i, l,a)) 
                    
    def predict(self, X):
        '''
        predict values
        '''
        y_hat = self.sess.run(self.prediction, feed_dict = {self.X: X})
        return y_hat
    
    def get_compare(self,prediction, target):
        '''
        compare predicted values with targeted values
        returns: <dict>, {'Profitable Trades': <int>, 'Lost_trades': <int>}
        '''
        profitable = np.argwhere(target == 1)[:,0]
        calculated_profitable = np.argwhere(np.round(prediction) == 1)[:,0]
        set_profitable = set(profitable)
        really_profitable_trades = set_profitable.intersection(calculated_profitable)
        lost = len(calculated_profitable) - len(really_profitable_trades)
        return {'Profitable Trades': len(really_profitable_trades), 'Lost_trades': lost}
        
   
    def variable_summaries(self, var):
        '''
        
        '''
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean', mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var)
            

In [6]:
X, Y = np.load('X_3600.npy'),np.load('Y_3600.npy'),
x, y = generate_random_dataset(X, Y)

In [7]:
rnn = NNModel()
rnn.fit(X,Y, epochs = 750000, every = 5000, generator = True)

2018-12-30 04:15:43,066 __main__ DEBUG Initializing


[ Sun Dec 30 04:15:43 2018 ] Epoch 0 Loss: 0.6782134 accuracy 0.48148149251937866
[ Sun Dec 30 04:19:05 2018 ] Epoch 5000 Loss: 0.6386988 accuracy 0.4444444477558136
[ Sun Dec 30 04:22:27 2018 ] Epoch 10000 Loss: 0.5939413 accuracy 0.7037037014961243
[ Sun Dec 30 04:25:47 2018 ] Epoch 15000 Loss: 0.6296215 accuracy 0.5925925970077515
[ Sun Dec 30 04:29:05 2018 ] Epoch 20000 Loss: 0.6319671 accuracy 0.6666666865348816
[ Sun Dec 30 04:32:21 2018 ] Epoch 25000 Loss: 0.6309261 accuracy 0.5925925970077515
[ Sun Dec 30 04:35:36 2018 ] Epoch 30000 Loss: 0.6462585 accuracy 0.5555555820465088
[ Sun Dec 30 04:38:55 2018 ] Epoch 35000 Loss: 0.6669331 accuracy 0.5925925970077515
[ Sun Dec 30 04:42:16 2018 ] Epoch 40000 Loss: 0.5976955 accuracy 0.6296296119689941
[ Sun Dec 30 04:45:33 2018 ] Epoch 45000 Loss: 0.5928024 accuracy 0.6666666865348816
[ Sun Dec 30 04:48:54 2018 ] Epoch 50000 Loss: 0.6152413 accuracy 0.6666666865348816
[ Sun Dec 30 04:52:16 2018 ] Epoch 55000 Loss: 0.5450385 accuracy 0.8

In [19]:
X_normalized = normalize_maxmin(X)
pred = rnn.predict(X_normalized)

In [38]:
rnn.sess.close()

In [35]:
def get_compare(prediction, target):
    profitable = np.argwhere(target == 1)[:,0]
    calculated_profitable = np.argwhere(np.round(prediction) == 1)[:,0]
    set_profitable = set(profitable)
    really_profitable_trades = set_profitable.intersection(calculated_profitable)
    lost = len(calculated_profitable) - len(really_profitable_trades)
    return {'Profitable Trades': len(really_profitable_trades), 'Lost_trades': lost}

In [11]:
saver.save(rnn.sess, 'Model/NN_FxData/model_13_00')

'Model/NN_FxData/model_13_00'

In [36]:
get_compare(pred, Y)

{'Profitable Trades': 24, 'Lost_trades': 14}

In [30]:
Y.shape

(314, 1)

In [31]:
target = Y
prediction = npred
profitable = np.argwhere(target == 1)[:,0]
calculated_profitable = np.argwhere(np.round(prediction == 1)[:,0]
set_profitable = set(profitable)
really_profitable_trades = set_profitable.intersection(calculated_profitable)
lost = len(calculated_profitable) - len(really_profitable_trades)

In [32]:
profitable

array([ 10,  33,  54,  63,  64,  79,  82,  83,  84,  85, 112, 142, 165,
       169, 171, 197, 226, 235, 238, 272, 273, 296, 300, 301])

In [33]:
calculated_profitable

array([ 10,  82,  83,  84,  85, 112, 142, 165, 169, 171, 197, 226, 235,
       238])